## Cài đặt môi trường

In [ ]:
!pip install -q flwr[simulation]
!pip install -q flwr-datasets[vision]

In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import random

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import Strategy
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

from flwr_datasets import FederatedDataset
from flwr_datasets.visualization import plot_label_distributions

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import DataLoader
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import DirichletPartitioner

import time


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

## Base config

In [ ]:
import os

CACHE_DIR = "/kaggle/working/hf_cache"
os.environ["HF_DATASETS_OFFLINE"] = "0"
os.makedirs(CACHE_DIR, exist_ok=True)

In [ ]:
backend_config = {"client_resources": {"num_cpus": 1}}
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1}}  # GPU optional

NUM_PARTITIONS = 20
NUM_ROUND = 1
EPOCH = 2
BATCH_SIZE = 20


FRACTION_FIT=0.85
FRACTION_EVALUATE=0.3
MIN_FIT_CLIENTS=7
MIN_EVALUATE_CLIENTS=10
MIN_AVAILABLE_CLIENTS=10

ALPHA = 0.3
CLIENT_LR_OPTIONS = [0.1, 0.01, 0.001, 10**(-2.5), 10**(-2.4)]
CURRENT_CLIENT_LR = None  # Sẽ được set trong loop

SERVER_LR = 3* 10**(-2)

# Tham số FedAdam
BETA_1 = 0.9
BETA_2 = 0.99
TAU = 1e-3  # Adaptive tau

## Load Dataset

In [ ]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)  # Chỉ cần gọi 1 lần ở đầu chương trình

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import DataLoader
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import DirichletPartitioner

# Cập nhật transforms cho CIFAR-10 với ResNet50
def load_datasets(partition_id, num_partitions: int):
    # Tạo dữ liệu Non-IID
    partitioner = DirichletPartitioner(
        num_partitions=NUM_PARTITIONS,
        partition_by="label",
        alpha=ALPHA,
        min_partition_size=num_partitions,
        self_balancing=True
    )

    fds = FederatedDataset(
        dataset="cifar10",
        partitioners={"train": partitioner},
        cache_dir=CACHE_DIR
    )

    partition = fds.load_partition(partition_id)
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)

    # Transforms được tối ưu cho ResNet18 + FL
    train_transforms = transforms.Compose([
        transforms.Resize((112, 112)),  # Nhỏ hơn, tiết kiệm computation
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=5),  # Giảm từ 10 xuống 5
        transforms.ColorJitter(brightness=0.1, contrast=0.1),  # Thêm augmentation nhẹ
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_transforms = transforms.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    def apply_train_transforms(batch):
        try:
            batch["img"] = [train_transforms(img.convert("RGB")) for img in batch["img"]]
            return batch
        except Exception as e:
            print(f"Error in train transforms: {e}")
            raise

    def apply_val_transforms(batch):
        try:
            batch["img"] = [val_transforms(img.convert("RGB")) for img in batch["img"]]
            return batch
        except Exception as e:
            print(f"Error in val transforms: {e}")
            raise

    # Apply transforms
    partition_train_test["train"] = partition_train_test["train"].with_transform(apply_train_transforms)
    partition_train_test["test"] = partition_train_test["test"].with_transform(apply_val_transforms)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pin_memory = torch.cuda.is_available()
    num_workers = 0  # Luôn để 0 để tránh bug multiprocessing

    trainloader = DataLoader(
        partition_train_test["train"],
        batch_size=BATCH_SIZE,
        shuffle=True,
        pin_memory=pin_memory,
        num_workers=num_workers,
        persistent_workers=False,
        drop_last=True
    )

    valloader = DataLoader(
        partition_train_test["test"],
        batch_size=BATCH_SIZE,
        shuffle=False,
        pin_memory=pin_memory,
        num_workers=num_workers,
        persistent_workers=False,
        drop_last=False
    )


    testset = fds.load_split("test").with_transform(apply_val_transforms)
    testloader = DataLoader(
        testset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        pin_memory=pin_memory,
        num_workers=num_workers,
        persistent_workers=False,  # num_workers=0 thì phải là False
        drop_last=False
    )

    return trainloader, valloader, testloader


In [ ]:
from huggingface_hub import login

login(token="")
all_data = [load_datasets(i, NUM_PARTITIONS) for i in range(NUM_PARTITIONS)]

In [ ]:
from flwr_datasets import FederatedDataset
from flwr_datasets.visualization import plot_label_distributions
import matplotlib.pyplot as plt

# Tạo partitioner
partitioner = DirichletPartitioner(
    num_partitions=NUM_PARTITIONS,
    partition_by="label",
    alpha=ALPHA,
    min_partition_size=NUM_PARTITIONS,
    self_balancing=True
)

# Tạo FederatedDataset
fds = FederatedDataset(
    dataset="cifar10",
    partitioners={"train": partitioner},
    cache_dir=CACHE_DIR
)

# Visualize phân bố
print(f"Visualizing Non-IID distribution with alpha={ALPHA}")
partitioner = fds.partitioners["train"]
figure, axis, dataframe = plot_label_distributions(
    partitioner=partitioner,
    label_name="label",
    legend=True,
    verbose_labels=True,
    plot_type="bar",  # Thêm này để rõ hơn
    size_unit="absolute"  # Hoặc "percentage"
)

# Customize plot
plt.title(f'CIFAR-10 Non-IID Distribution (α={ALPHA})')
plt.xlabel('Clients')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# In thống kê
print("\n=== Distribution Statistics ===")
print(f"Total clients: {NUM_PARTITIONS}")
print(f"Alpha (non-IID level): {ALPHA}")
print(f"Dataset: CIFAR-10")
print(f"Classes: {dataframe.columns.tolist()}")

# Thống kê chi tiết
total_samples = dataframe.sum().sum()
print(f"Total samples: {total_samples}")
print(f"Adam samples per client: {total_samples/NUM_PARTITIONS:.1f}")
print(f"Min samples: {dataframe.sum(axis=1).min()}")
print(f"Max samples: {dataframe.sum(axis=1).max()}")

In [ ]:
import numpy as np
from scipy.stats import entropy

# Function to calculate entropy for a list of counts
def calculate_entropy(counts):
    total = sum(counts)
    if total == 0:
        return 0
    probabilities = [c / total for c in counts]
    return entropy(probabilities, base=2) # Using base 2 for bits

entropies = []
for client_id in range(NUM_PARTITIONS):
    label_counts = dataframe.iloc[client_id].tolist() # Get label counts for this client
    entropies.append(calculate_entropy(label_counts))

average_entropy = np.mean(entropies)
std_entropy = np.std(entropies)

print(f"Average Entropy: {average_entropy:.4f}")
print(f"Entropy Std: {std_entropy:.4f}")


## Model ResNet18

In [ ]:
import torch.nn as nn
from torchvision import models

class ResNet18_GroupNorm(nn.Module):
    def __init__(self, num_classes: int = 10, pretrained: bool = True):
        super(ResNet18_GroupNorm, self).__init__()

        # Sử dụng pretrained weights
        weights = models.ResNet18_Weights.IMAGENET1K_V1 if pretrained else None
        self.backbone = models.resnet18(weights=weights)

        # Thay BatchNorm bằng GroupNorm (2 nhóm/lớp)
        self._replace_batchnorm_with_groupnorm(self.backbone)

        # Lấy số feature từ fc gốc
        feat_dim = self.backbone.fc.in_features

        # Bỏ lớp fc gốc
        self.backbone.fc = nn.Identity()

        # Classifier với GroupNorm
        self.classifier = nn.Sequential(
            # Thêm dropout để tránh overfitting
            nn.Dropout(0.3),  # Dropout sau feature extraction
            nn.Linear(feat_dim, 128),
            nn.GroupNorm(2, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),  # Dropout giữa các lớp

            nn.Linear(128, 64),
            nn.GroupNorm(2, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),  # Dropout nhẹ hơn ở cuối

            nn.Linear(64, num_classes)
        )

        self._init_weights()

    def _replace_batchnorm_with_groupnorm(self, module):
        """Thay tất cả BatchNorm bằng GroupNorm với 2 nhóm"""
        for name, child in module.named_children():
            if isinstance(child, nn.BatchNorm2d):
                num_channels = child.num_features
                # Thay BatchNorm2d bằng GroupNorm với 2 nhóm
                setattr(module, name, nn.GroupNorm(2, num_channels))
            elif isinstance(child, nn.BatchNorm1d):
                num_channels = child.num_features
                # Thay BatchNorm1d bằng GroupNorm với 2 nhóm
                setattr(module, name, nn.GroupNorm(2, num_channels))
            else:
                # Đệ quy cho các module con
                self._replace_batchnorm_with_groupnorm(child)

    def _init_weights(self):
        for m in self.classifier:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.backbone(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# =====================================
# 3. CẬP NHẬT HÀM TRAIN VỚI CLIENT_LR
# =====================================

def train(net, trainloader, epochs: int):
    """Huấn luyện mô hình với client learning rate từ grid search"""
    criterion = torch.nn.CrossEntropyLoss()
    # Sử dụng CURRENT_CLIENT_LR thay vì CLIENT_LR cố định
    optimizer = torch.optim.SGD(net.parameters(), lr=CURRENT_CLIENT_LR, weight_decay=1e-3)
    net.train()

    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * labels.size(0)
            total += labels.size(0)
            correct += (outputs.argmax(dim=1) == labels).sum().item()

        epoch_loss /= total
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss:.4f}, accuracy {epoch_acc:.4f}")

    return epoch_loss, epoch_acc

def test(net, testloader):
    """Đánh giá mô hình trên tập validation và trả về loss, acc."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, total_loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * labels.size(0)
            correct += (outputs.argmax(dim=1) == labels).sum().item()
            total += labels.size(0)
    adam_loss = total_loss / total
    accuracy = correct / total
    return adam_loss, accuracy


In [ ]:
def get_parameters(net) -> List[np.ndarray]:
    params = []
    for name, val in net.state_dict().items():
        if val.numel() > 0:
            params.append(val.cpu().numpy())
        else:
            print(f"Warning: Empty tensor found: {name}")
    return params

def set_parameters(net, parameters: List[np.ndarray]):
    if not parameters:
        print("Warning: No parameters received")
        return

    state_dict_keys = list(net.state_dict().keys())
    print(f"Model has {len(state_dict_keys)} parameters")
    print(f"Received {len(parameters)} parameters")

    if len(parameters) != len(state_dict_keys):
        print("Parameter count mismatch!")
        return

    params_dict = zip(state_dict_keys, parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=False)

## Custom FedAdam

In [ ]:
class FlowerClientFedAdam(NumPyClient):
    def __init__(self, partition_id, net, trainloader, valloader):
        self.partition_id = partition_id
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.partition_id}] FedAdam fit")
        set_parameters(self.net, parameters)
        loss, acc = train(self.net, self.trainloader, epochs=EPOCH)
        return get_parameters(self.net), len(self.trainloader.dataset), {
            "loss": float(loss), "accuracy": float(acc)
        }

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {
            "loss": float(loss), "accuracy": float(accuracy)
        }

In [ ]:
def client_fn_fedadam(context):
    cid = context.node_config["partition-id"]
    trainloader, valloader, _ = all_data[cid]

    net = ResNet18_GroupNorm(num_classes=10).to(DEVICE)
    return FlowerClientFedAdam(cid, net, trainloader, valloader).to_client()

client_app_fedadam = ClientApp(client_fn=client_fn_fedadam)

In [ ]:
from flwr.common import Metrics, Context
from flwr.common import ndarrays_to_parameters

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = ResNet18_GroupNorm().to(DEVICE)
initial_parameters = flwr.common.ndarrays_to_parameters(get_parameters(net))

In [ ]:
from flwr.server.strategy import FedAdam
import time

class CustomFedAdam(FedAdam):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.metrics_centralized = []
        self.round_start_times = {}

    def configure_fit(self, server_round, parameters, client_manager):
        # Ghi lại thời gian bắt đầu của round
        self.round_start_times[server_round] = time.time()
        return super().configure_fit(server_round, parameters, client_manager)

    def aggregate_fit(self, rnd, results, failures):
        aggregated_parameters, _ = super().aggregate_fit(rnd, results, failures)

        losses = [r.metrics.get("loss") for _, r in results if "loss" in r.metrics]
        accs = [r.metrics.get("accuracy") for _, r in results if "accuracy" in r.metrics]
        avg_loss = sum(losses) / len(losses) if losses else None
        avg_acc = sum(accs) / len(accs) if accs else None

        print(f"[Round {rnd}] Train — Loss: {avg_loss:.4f}, Acc: {avg_acc:.4f}" if avg_loss is not None else
              f"[Round {rnd}] Train — Loss: N/A, Acc: N/A")

        while len(self.metrics_centralized) < rnd:
            self.metrics_centralized.append({})

        self.metrics_centralized[rnd - 1].update({
            "train_loss": avg_loss,
            "train_acc": avg_acc
        })

        return aggregated_parameters, {}

    def aggregate_evaluate(self, rnd, results, failures):
        eval_start = time.time()
        aggregated_loss, _ = super().aggregate_evaluate(rnd, results, failures)
        eval_duration = time.time() - eval_start

        round_total_time = time.time() - self.round_start_times.get(rnd, time.time())

        losses = [r.metrics.get("loss") for _, r in results if "loss" in r.metrics]
        accs = [r.metrics.get("accuracy") for _, r in results if "accuracy" in r.metrics]
        avg_loss = sum(losses) / len(losses) if losses else None
        avg_acc = sum(accs) / len(accs) if accs else None

        print(f"[Round {rnd}] Val   — Loss: {avg_loss:.4f}, Acc: {avg_acc:.4f}, Eval Time: {eval_duration:.2f}s")
        print(f"[Round {rnd}] Total Round Time: {round_total_time:.2f}s")

        while len(self.metrics_centralized) < rnd:
            self.metrics_centralized.append({})

        self.metrics_centralized[rnd - 1].update({
            "val_loss": avg_loss,
            "val_acc": avg_acc,
            "eval_duration": eval_duration,
            "round_total_time": round_total_time
        })

        return aggregated_loss, {}


In [ ]:
class CustomFedAdamGridSearch(FedAdam):
    def __init__(self, client_lr, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_lr = client_lr
        self.metrics_centralized = []
        self.round_start_times = {}

    def configure_fit(self, server_round, parameters, client_manager):
        self.round_start_times[server_round] = time.time()
        return super().configure_fit(server_round, parameters, client_manager)

    def aggregate_fit(self, rnd, results, failures):
        aggregated_parameters, _ = super().aggregate_fit(rnd, results, failures)

        losses = [r.metrics.get("loss") for _, r in results if "loss" in r.metrics]
        accs = [r.metrics.get("accuracy") for _, r in results if "accuracy" in r.metrics]
        avg_loss = sum(losses) / len(losses) if losses else None
        avg_acc = sum(accs) / len(accs) if accs else None

        print(f"[LR={self.client_lr:.6f}][Round {rnd}] Train — Loss: {avg_loss:.4f}, Acc: {avg_acc:.4f}" if avg_loss is not None else
              f"[LR={self.client_lr:.6f}][Round {rnd}] Train — Loss: N/A, Acc: N/A")

        while len(self.metrics_centralized) < rnd:
            self.metrics_centralized.append({})

        self.metrics_centralized[rnd - 1].update({
            "train_loss": avg_loss,
            "train_acc": avg_acc,
            "client_lr": self.client_lr
        })

        return aggregated_parameters, {}

    def aggregate_evaluate(self, rnd, results, failures):
        eval_start = time.time()
        aggregated_loss, _ = super().aggregate_evaluate(rnd, results, failures)
        eval_duration = time.time() - eval_start
        round_total_time = time.time() - self.round_start_times.get(rnd, time.time())

        losses = [r.metrics.get("loss") for _, r in results if "loss" in r.metrics]
        accs = [r.metrics.get("accuracy") for _, r in results if "accuracy" in r.metrics]
        avg_loss = sum(losses) / len(losses) if losses else None
        avg_acc = sum(accs) / len(accs) if accs else None

        print(f"[LR={self.client_lr:.6f}][Round {rnd}] Val — Loss: {avg_loss:.4f}, Acc: {avg_acc:.4f}, Time: {round_total_time:.2f}s")

        while len(self.metrics_centralized) < rnd:
            self.metrics_centralized.append({})

        self.metrics_centralized[rnd - 1].update({
            "val_loss": avg_loss,
            "val_acc": avg_acc,
            "eval_duration": eval_duration,
            "round_total_time": round_total_time,
            "client_lr": self.client_lr
        })

        return aggregated_loss, {}

In [ ]:
def run_grid_search():
    """Chạy grid search cho CLIENT_LR"""
    global CURRENT_CLIENT_LR

    all_results = {}
    best_lr = None
    best_val_acc = 0

    print("🔍 Starting Grid Search for CLIENT_LR...")
    print(f"Testing LR values: {CLIENT_LR_OPTIONS}")
    print(f"Running {NUM_ROUND} rounds per experiment")
    print("="*60)

    for i, lr in enumerate(CLIENT_LR_OPTIONS):
        print(f"\n🧪 Experiment {i+1}/{len(CLIENT_LR_OPTIONS)}: CLIENT_LR = {lr}")
        print("-" * 50)

        # Set current learning rate
        CURRENT_CLIENT_LR = lr

        # Tạo fresh model cho mỗi experiment
        net = ResNet18_GroupNorm().to(DEVICE)
        initial_parameters = flwr.common.ndarrays_to_parameters(get_parameters(net))

        # Tạo strategy cho experiment này
        strategy = CustomFedAdamGridSearch(
            client_lr=lr,
            fraction_fit=FRACTION_FIT,
            fraction_evaluate=FRACTION_EVALUATE,
            min_fit_clients=MIN_FIT_CLIENTS,
            min_evaluate_clients=MIN_EVALUATE_CLIENTS,
            min_available_clients=MIN_AVAILABLE_CLIENTS,
            evaluate_metrics_aggregation_fn=weighted_average,
            initial_parameters=initial_parameters,
            eta=SERVER_LR,
            beta_1=BETA_1,
            beta_2=BETA_2,
            tau=TAU,
        )

        def server_fn_gs(context: Context) -> ServerAppComponents:
            config = ServerConfig(num_rounds=NUM_ROUND)
            return ServerAppComponents(strategy=strategy, config=config)

        server_gs = ServerApp(server_fn=server_fn_gs)

        # Chạy simulation
        experiment_start = time.time()
        run_simulation(
            server_app=server_gs,
            client_app=client_app_fedadam,
            num_supernodes=NUM_PARTITIONS,
            backend_config=backend_config,
        )
        experiment_duration = time.time() - experiment_start

        # Lưu results
        all_results[lr] = {
            "strategy": strategy,
            "final_val_acc": strategy.metrics_centralized[-1].get("val_acc", 0),
            "experiment_duration": experiment_duration
        }

        # Track best result
        final_val_acc = strategy.metrics_centralized[-1].get("val_acc", 0)
        if final_val_acc > best_val_acc:
            best_val_acc = final_val_acc
            best_lr = lr

        print(f"✅ Completed LR={lr}: Final Val Acc = {final_val_acc:.4f}, Duration = {experiment_duration/60:.1f}min")

    print("\n" + "="*60)
    print("🎯 GRID SEARCH COMPLETED!")
    print(f"🏆 Best CLIENT_LR: {best_lr} (Val Acc: {best_val_acc:.4f})")
    print("="*60)

    return all_results, best_lr

In [ ]:
# Chạy grid search
grid_search_results, best_client_lr = run_grid_search()


In [ ]:
save_dir = "fl_results_gs_cl"
def save_results_to_json_unified(strategy, strategy_name, save_dir=save_dir):
    """
    Lưu kết quả huấn luyện với định dạng JSON thống nhất.
    Bao gồm train/val loss, acc, thời gian evaluate và tổng thời gian round.
    """
    os.makedirs(save_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    unified_metrics = []
    for i, metrics in enumerate(strategy.metrics_centralized):
        unified_metrics.append({
            "round": i + 1,
            "train_loss": metrics.get("train_loss"),
            "train_acc": metrics.get("train_acc"),
            "val_loss": metrics.get("val_loss"),
            "val_acc": metrics.get("val_acc"),
            "eval_duration": metrics.get("eval_duration"),
            "round_total_time": metrics.get("round_total_time", metrics.get("round_duration"))
        })

    round_times = [m["round_total_time"] for m in unified_metrics if m["round_total_time"] is not None]
    total_runtime = sum(round_times)
    avg_runtime = total_runtime / len(round_times) if round_times else 0

    results = {
        "strategy_name": strategy_name,
        "timestamp": timestamp,
        "metrics": unified_metrics,
        "runtime_summary": {
            "total_runtime_seconds": total_runtime,
            "average_round_time_seconds": avg_runtime,
            "fastest_round_seconds": min(round_times) if round_times else None,
            "slowest_round_seconds": max(round_times) if round_times else None,
            "num_rounds": len(unified_metrics)
        },
        "config": {
            "num_rounds": len(unified_metrics),
            "num_partitions": globals().get("NUM_PARTITIONS", None),
            "batch_size": globals().get("BATCH_SIZE", None),
            "learning_rate": globals().get("CLIENT_LR", None),
            "epochs_per_round": globals().get("EPOCH", None)
        }
    }

    filename = f"{save_dir}/{strategy_name}_{timestamp}.json"
    with open(filename, "w") as f:
        json.dump(results, f, indent=2)

    print(f"✅ Saved unified JSON for {strategy_name} to {filename}")
    return filename


In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
import os
import json

def plot_grid_search_results(results):
    """Vẽ biểu đồ so sánh các learning rate"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    colors = ['blue', 'red', 'green', 'orange', 'purple']

    for i, (lr, data) in enumerate(results.items()):
        strategy = data["strategy"]
        metrics = strategy.metrics_centralized
        rounds = list(range(1, len(metrics) + 1))

        train_acc = [m.get("train_acc") for m in metrics]
        val_acc = [m.get("val_acc") for m in metrics]
        train_loss = [m.get("train_loss") for m in metrics]
        val_loss = [m.get("val_loss") for m in metrics]

        color = colors[i % len(colors)]
        label = f"LR={lr:.6f}" if lr < 0.01 else f"LR={lr}"

        # Train Accuracy
        axes[0,0].plot(rounds, train_acc, label=label, color=color, marker='o', markersize=3)
        # Val Accuracy
        axes[0,1].plot(rounds, val_acc, label=label, color=color, marker='s', markersize=3)
        # Train Loss
        axes[1,0].plot(rounds, train_loss, label=label, color=color, marker='^', markersize=3)
        # Val Loss
        axes[1,1].plot(rounds, val_loss, label=label, color=color, marker='v', markersize=3)

    axes[0,0].set_title("Training Accuracy")
    axes[0,0].set_xlabel("Round")
    axes[0,0].set_ylabel("Accuracy")
    axes[0,0].legend()
    axes[0,0].grid(True)

    axes[0,1].set_title("Validation Accuracy")
    axes[0,1].set_xlabel("Round")
    axes[0,1].set_ylabel("Accuracy")
    axes[0,1].legend()
    axes[0,1].grid(True)

    axes[1,0].set_title("Training Loss")
    axes[1,0].set_xlabel("Round")
    axes[1,0].set_ylabel("Loss")
    axes[1,0].legend()
    axes[1,0].grid(True)

    axes[1,1].set_title("Validation Loss")
    axes[1,1].set_xlabel("Round")
    axes[1,1].set_ylabel("Loss")
    axes[1,1].legend()
    axes[1,1].grid(True)

    plt.suptitle("Grid Search Results: CLIENT_LR Comparison", fontsize=16)
    plt.tight_layout()
    plt.show()


def save_grid_search_results(results, best_lr, save_dir =save_dir):

    """Lưu kết quả grid search"""
    import pandas as pd
    os.makedirs(save_dir, exist_ok=True)
    # Tạo summary table
    summary_data = []
    for lr, data in results.items():
        strategy = data["strategy"]
        metrics = strategy.metrics_centralized

        final_train_acc = metrics[-1].get("train_acc", 0)
        final_val_acc = metrics[-1].get("val_acc", 0)
        final_train_loss = metrics[-1].get("train_loss", 0)
        final_val_loss = metrics[-1].get("val_loss", 0)

        best_val_acc = max([m.get("val_acc", 0) for m in metrics])
        best_round = [m.get("val_acc", 0) for m in metrics].index(best_val_acc) + 1

        summary_data.append({
            "client_lr": lr,
            "final_train_acc": final_train_acc,
            "final_val_acc": final_val_acc,
            "final_train_loss": final_train_loss,
            "final_val_loss": final_val_loss,
            "best_val_acc": best_val_acc,
            "best_val_round": best_round,
            "experiment_duration_min": data["experiment_duration"] / 60,
            "is_best": lr == best_lr
        })

    df = pd.DataFrame(summary_data)
    df = df.sort_values("final_val_acc", ascending=False)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    print("\n📊 GRID SEARCH SUMMARY:")
    print(df.to_string(index=False))
    # Lưu từng strategy riêng lẻ
    for lr, data in results.items():
        strategy_name = f"FedAdam_LR_{lr:.6f}".replace(".", "_")
        save_results_to_json_unified(data["strategy"], strategy_name)

In [ ]:
# Vẽ biểu đồ và lưu kết quả
plot_grid_search_results(grid_search_results)
save_grid_search_results(grid_search_results, best_client_lr)